<a href="https://colab.research.google.com/github/abyanjan/Natural-language-Processing-NLP-/blob/master/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modelling Using LDA(Latent Dirichlet Allocation)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.stem.porter import PorterStemmer


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Data
df = pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true',compression='zip',sep=',',quotechar='"')

In [4]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt was incurred due to medical malpract...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,None,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704


Data is related to customer complaints made by customers in different services related to the banks.

In [5]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [6]:
df['Company'].value_counts()

CITIBANK, N.A.                                                 3226
CAPITAL ONE FINANCIAL CORPORATION                              2711
BANK OF AMERICA, NATIONAL ASSOCIATION                          2580
JPMORGAN CHASE & CO.                                           2409
WELLS FARGO & COMPANY                                          2001
                                                               ... 
Rhodes, Kelly Assoc Inc                                           1
Miller and Steeno, P.C.                                           1
RABOBANK, NATIONAL ASSOCIATION                                    1
KMD Partners, LLC                                                 1
Citizens Savings & Loan Corporation, Chattanooga, TN Branch       1
Name: Company, Length: 2197, dtype: int64

In [7]:
# selecting the columns and creating a new data
complaint_df = df[['Consumer complaint narrative','Product','Company']].rename(columns = {'Consumer complaint narrative':'complaints'})

In [8]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [10]:
complaint_df.head()

,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

**Train Test Split**

In [9]:
X_train, X_test = train_test_split(complaint_df, test_size = 0.5, random_state = 11)

In [11]:
X_train.Product.value_counts()

Debt collection                10882
Credit card or prepaid card    6551 
Mortgage                       4873 
Checking or savings account    3527 
Student loan                   1512 
Vehicle loan or lease          1381 
Name: Product, dtype: int64

**Data Processing**

In [12]:
# initialize stemming
stemmer = PorterStemmer()

In [43]:
# word tokenization function
def tokenize(text):
  tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2 and len(re.sub('\d+','', word.strip('Xx/'))) > 2)]
  #stems = [stemmer.stem(word) for word in tokens]
  return tokens

In [44]:
# count vectorizer
vectorizer = CountVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, min_df=50, max_features=10000)

In [ ]:
tf_vectors = vectorizer.fit_transform(X_train['complaints'])

In [47]:
tf_vectors.A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [48]:
vectorizer.get_feature_names()[1:20]

['abide',
 'ability',
 'able',
 'above-named',
 'abruptly',
 'absolute',
 'absolutely',
 'absurd',
 'abuse',
 'abused',
 'abusive',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'access',
 'accessed',
 'accessible']

**Topic Modelling - LDA**

In [51]:
lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=6,learning_method='online', learning_offset=50,n_jobs=-1, random_state=111)

In [52]:
# probability of a document belonging to the 6 topics
topic_weights = lda.fit_transform(tf_vectors)
topic_weights

array([[0.00525585, 0.15198299, 0.0052432 , 0.00529711, 0.82699261,
        0.00522824],
       [0.91524126, 0.0016278 , 0.03731678, 0.04255544, 0.00162629,
        0.00163243],
       [0.30314044, 0.08648794, 0.03108877, 0.14013567, 0.334871  ,
        0.10427617],
       ...,
       [0.00397571, 0.00398046, 0.00398616, 0.20959967, 0.00398123,
        0.77447677],
       [0.00669457, 0.00673135, 0.96642074, 0.00669593, 0.00676003,
        0.00669737],
       [0.18788646, 0.0033641 , 0.55717786, 0.00339308, 0.00337781,
        0.24480068]])

In [53]:
# lda components
components = lda.components_

In [57]:
# taking top 15 words from each topic
num_words = 15

# vocabulary of all the words
vocab = np.array(vectorizer.get_feature_names())

# function to extract top words
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]

topic_words = ([top_words(t) for t in components])
topic_words = [" ".join(t) for t in topic_words]

In [58]:
topic_words

['told called said phone asked time company number money just received know help times calls',
 'account bank check chase money funds checking received days customer wells america fargo closed online',
 'debt credit account collection information report company letter reporting sent received provide agency original consumer',
 'mortgage loan home insurance property received escrow letter sent documents company modification closing servicing information',
 'card credit account charges charge dispute capital fraud report citi cards fraudulent closed balance purchase',
 'payment payments loan late paid balance credit month fees account time loans monthly make months']

**Adding the topics to the data**

In [62]:
colnames = ['Topic'+str(i) for i in range(lda.n_components)]
documents = ['Doc'+str(i) for i in range(len(X_train.complaints))]

df_doc_topic = pd.DataFrame(np.round(topic_weights,2), columns=colnames, index=documents)

top_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['Dominant_topic'] = top_topic

In [63]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant_topic
Doc0,0.01,0.15,0.01,0.01,0.83,0.01,4
Doc1,0.92,0.00,0.04,0.04,0.00,0.00,0
Doc2,0.30,0.09,0.03,0.14,0.33,0.10,4
Doc3,0.44,0.04,0.00,0.50,0.00,0.00,3
Doc4,0.00,0.00,0.98,0.00,0.00,0.00,2
...,...,...,...,...,...,...,...
Doc28721,0.00,0.98,0.00,0.00,0.00,0.00,1
Doc28722,0.30,0.57,0.07,0.00,0.00,0.06,1
Doc28723,0.00,0.00,0.00,0.21,0.00,0.77,5
Doc28724,0.01,0.01,0.97,0.01,0.01,0.01,2


**Applying the topic modeling to test data**

In [66]:
# testing for first 5 rows
test_topic_weights = lda.transform(vectorizer.transform(X_test.complaints[:5]))

In [67]:
colnames = ['Topic'+str(i) for i in range(lda.n_components)]
documents = ['Doc'+str(i) for i in range(len(X_test.complaints[:5]))]

df_doc_topic = pd.DataFrame(np.round(test_topic_weights,2), columns=colnames, index=documents)

top_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['Dominant_topic'] = top_topic

In [75]:
X_test[['complaints','Product']].head(5)

complaints                      Product
50868  I.C. Systems has reported and posted to my credit that I owe {$550.00} for a XXXX  bill. I contacted XXXX  and they stated the amount owed is {$320.00}. I have called both XXXX  and I.C. Systems multiple times since XX/XX/2020 -today XX/XX/2020 trying to get the correct balance updated so the debt can be paid. I.C. Systems have yet to make any attempt to update this amount.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [68]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant_topic
Doc0,0.16,0.01,0.50,0.01,0.01,0.31,2
Doc1,0.01,0.01,0.01,0.01,0.01,0.96,5
Doc2,0.63,0.12,0.00,0.03,0.16,0.05,0
Doc3,0.00,0.00,0.00,0.00,0.98,0.00,4
Doc4,0.28,0.03,0.14,0.19,0.37,0.00,4


In [73]:
topic_words

['told called said phone asked time company number money just received know help times calls',
 'account bank check chase money funds checking received days customer wells america fargo closed online',
 'debt credit account collection information report company letter reporting sent received provide agency original consumer',
 'mortgage loan home insurance property received escrow letter sent documents company modification closing servicing information',
 'card credit account charges charge dispute capital fraud report citi cards fraudulent closed balance purchase',
 'payment payments loan late paid balance credit month fees account time loans monthly make months']

We can now compare if the top words in the topic the text was assigned to matches the text description. From the comparison on the first 5 test rows, it seems that the topic modelling is doing a decent job in assigining topic to a text.